In [57]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

import tqdm
import os
import mcbe
from mpl_toolkits.mplot3d import Axes3D

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

X_3 = X[:,[0,2,3]]

# Scale data to have mean 0 and variance 1 
# which is importance for convergence of the neural network
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_3)
#m = np.amax(X_scaled)
#X_ball = X_scaled/m

# Split the data set into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

In [59]:
def homo(A,filename = 'Test',name='w1',epoch=1):
    np.set_printoptions(precision=4,threshold=10_000,suppress = True)
    if torch.is_tensor(A):
        A = A.detach().numpy()
    h = np.ones((A.shape[0],1))
    A_homo = np.concatenate((h,A), axis = 1)
    mat = np.matrix(A_homo)
    with open(filename+'/'+name+'_ep'+str(epoch)+'.txt','wb') as f:
        for line in mat:
            np.savetxt(f, line, fmt='%.2f')
    
    return

In [60]:
# size of the first layer
l1 = 200

In [61]:
class Model(nn.Module):
    def __init__(self, input_dim, l1):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, l1)
        #self.layer2 = nn.Linear(20, 10)
        self.layer3 = nn.Linear(l1, 3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        #x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

In [62]:
model     = Model(X_train.shape[1],l1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn   = nn.CrossEntropyLoss()
model

Model(
  (layer1): Linear(in_features=3, out_features=200, bias=True)
  (layer3): Linear(in_features=200, out_features=3, bias=True)
)

In [63]:
#initialize lists

EPOCHS  = 25

weights = []
weights_norm = []
norms = []
biases = np.zeros([EPOCHS + 1,l1])

w1 = model.layer1.weight
b1 = model.layer1.bias
m1 = w1.shape[0]
n1 = w1.shape[1]
norm = w1.pow(2).sum(keepdim=True,dim=1).sqrt()
w1_norm = torch.div(w1,norm)
w1_norm[w1_norm == np.inf] = 0

    
weights.append(w1.detach().numpy())
weights_norm.append(w1_norm.detach().numpy())
norms.append(norm.detach().numpy())
biases[0,:] = b1.detach().numpy()

#epoch counter
k = 1

X_train = Variable(torch.from_numpy(X_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()
X_test  = Variable(torch.from_numpy(X_test)).float()
y_test  = Variable(torch.from_numpy(y_test)).long()

loss_list     = np.zeros((EPOCHS,))
accuracy_list = np.zeros((EPOCHS,))

for epoch in tqdm.trange(EPOCHS):
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    loss_list[epoch] = loss.item()
    
    # Zero gradients
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        y_pred = model(X_test)
        correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.FloatTensor)
        accuracy_list[epoch] = correct.mean()
    
    
    
    w1 = model.layer1.weight
    b1 = model.layer1.bias
    norm = w1.pow(2).sum(keepdim=True,dim=1).sqrt()
    w1_norm = torch.div(w1,norm)
    w1_norm[w1_norm == np.inf] = 0
    
    
    
    weights.append(w1.detach().numpy())
    weights_norm.append(w1_norm.detach().numpy())
    norms.append(norm.detach().numpy())
    biases[k,:] = b1.detach().numpy()
    
    k = k+1
    


accuracy_list

100%|██████████| 25/25 [00:00<00:00, 659.68it/s]


array([0.80000001, 0.76666665, 0.76666665, 0.83333331, 0.83333331,
       0.83333331, 0.83333331, 0.83333331, 0.93333334, 0.93333334,
       0.89999998, 0.93333334, 0.89999998, 0.89999998, 0.93333334,
       0.96666664, 0.96666664, 0.96666664, 0.96666664, 0.96666664,
       0.96666664, 0.96666664, 0.96666664, 0.96666664, 0.96666664])

# evaluation

In [64]:
num_evaluation_iter = 100
times_to_blowup = 2

## evaluate blowup

In [65]:
percent_inj_blowup_list = []

for iter in tqdm.tqdm(range(num_evaluation_iter)):
    
    X_train_noisy = X_train + np.random.normal(0,0.5,X_train.shape)
    for i in range(times_to_blowup-1):
        X_train_noisy = torch.cat((X_train_noisy,X_train + np.random.normal(0,0.5,X_train.shape)),0)
        
    # rowbind the original data and the noisy data by adding the noisy data to the original data
    X_train_blowup = torch.cat((X_train,X_train_noisy),0)

    est_alpha_blowup, subframes_blowup, est_points_blowup = mcbe.be_given_points(polytope=np.array(weights)[-1],points=X_train_blowup, give_subframes=True)

    percent_inj_blowup = mcbe.check_injectivity_naive(W = np.array(weights)[-1], b=est_alpha_blowup, points=X_test,iter=X_test.shape[0])

    percent_inj_blowup_list.append(percent_inj_blowup)

 81%|████████  | 81/100 [01:29<00:21,  1.11s/it]


KeyboardInterrupt: 

## evaluate random added

In [ ]:
percent_inj_rd_added = []

for iter in tqdm.tqdm(range(num_evaluation_iter)):

    X_train_rd = np.random.normal(0,1,X_train.shape)
    X_train_rd = torch.from_numpy(X_train_rd).float()
    for i in range(times_to_blowup-1):
        X_train_rd = torch.cat((X_train_rd, torch.from_numpy(np.random.normal(0,1,X_train.shape))),0)
        
    # rowbind the original data and the noisy data by adding the noisy data to the original data
    X_train_add_rd = torch.cat((X_train,X_train_rd),0)

    est_alpha_rd, subframes_rd, est_points_rd = mcbe.be_given_points(polytope=np.array(weights)[-1],points=X_train_add_rd, give_subframes=True)

    percent_inj_rd = mcbe.check_injectivity_naive(W = np.array(weights)[-1], b=est_alpha_rd, points=X_test,iter=X_test.shape[0])

    percent_inj_rd_added.append(percent_inj_rd)


100%|██████████| 100/100 [01:55<00:00,  1.15s/it]


## evaluate standard mcbe

In [ ]:
percent_inj_mcbe_list = []
for iter in tqdm.tqdm(range(num_evaluation_iter)):

    est_alpha, subframes, est_points =mcbe.mcbe(polytope=np.array(weights)[-1],N=X_train_blowup.shape[0],distribution="ball",radius=np.max(np.array(X_train)), give_subframes=True,sample_on_sphere=False)

    percent_inj_mcbe = mcbe.check_injectivity_naive(W = np.array(weights)[-1], b=est_alpha, points=X_test,iter=X_test.shape[0])

    percent_inj_mcbe_list.append(percent_inj_mcbe)



100%|██████████| 100/100 [00:23<00:00,  4.22it/s]


## comparison

In [ ]:
np.mean(percent_inj_blowup_list)

1.0

In [ ]:
np.mean(percent_inj_rd_added)

1.0

In [ ]:
np.mean(percent_inj_mcbe_list)

0.7996666666666666